In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
from PIL import Image, ImageDraw
import requests
from pycocotools.coco import COCO
from pathlib import Path
import cv2
import torch
import os
import sys
from torch import nn, Tensor
# from torchvision import transforms
import torchvision
# from torchvision.transforms import functional as F
# from torchvision.transforms import transforms as T
import mrcnn

# Set the ROOT_DIR variable to the root directory of the Mask_RCNN git repo
ROOT_DIR = './Mask_RCNN'
assert os.path.exists(ROOT_DIR), 'ROOT_DIR does not exist. Did you forget to read the instructions above? ;)'

# Import mrcnn libraries
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
# import mrcnn.utils as utils
from mrcnn import visualize
# import mrcnn.model as modellib
from torchvision.ops import box_convert
import albumentations
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

In [2]:
import wandb
# wandb.init(project="my-test-project", entity="kolor200")

In [3]:
# wandb.finish()

In [4]:
# https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

# Composes transforms 
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

# List of transformations:

# Transforms data into torch Tensors
class ToTensor(nn.Module):
    def forward(self, image, target):
#         print(image.dtype, image.shape)
        image = torchvision.transforms.functional.pil_to_tensor(image)
        image = torchvision.transforms.functional.convert_image_dtype(image)
#         image = torchvision.transforms.functional.convert_image_dtype(image)
        
        return image, target

# Not needed performed by transformation seen above
class ConvertImageDtype(nn.Module):
    def __init__(self, dtype) -> None:
        super().__init__()
        self.dtype = dtype

    def forward(self, image, target):
        image = torchvision.transforms.functional.convert_image_dtype(image, self.dtype)
        return image, target

# Random Horizontal Flip
class RandomHorizontalFlip(torchvision.transforms.transforms.RandomHorizontalFlip):
    def forward(self, image, target):
        if torch.rand(1) < self.p:
            image = torchvision.transforms.functional.hflip(image)
            if target is not None:
                width, _ = torchvision.transforms.functional.get_image_size(image)
                target["boxes"][:, [0, 2]] = width - target["boxes"][:, [2, 0]]
                if "masks" in target:
                    target["masks"] = target["masks"].flip(-1)
        return image, target
    

class RandomImageEqualization(torchvision.transforms.transforms.RandomEqualize):
    def forward(self, image, target):
        if torch.rand(1) < self.p:
            image = torchvision.transforms.functional.equalize(image)
        return image, target


class RandomGaussianBlur(torchvision.transforms.transforms.RandomEqualize):
    def forward(self, image, target):
        if torch.rand(1) < self.p:
            image = torchvision.transforms.functional.gaussian_blur(image, kernel_size=(5, 9), sigma=(0.1, 5))
        return image, target

# This function applies random rotation, using wonderful library albumentations.ai
class RandomRotation(nn.Module):
    def __init__(self, p) -> None:
        super().__init__()
        self.p = p
        
    def forward(self, image, target):
        if torch.rand(1) < self.p:
            
            transform = albumentations.Compose([
                albumentations.Rotate(p=0.5, limit=(-90, 90), mask_value=0)
            ], bbox_params=albumentations.BboxParams(format='pascal_voc',  
                                                     min_area=1024, 
                                                     min_visibility=0.1))
              
            image = np.asarray(image)
            list_of_masks = []
            for mask in target["masks"]:
                list_of_masks.append(np.asarray(mask))
            bboxes = list(target["boxes"])
            
            class_labels = list(np.asarray(target["labels"]))
            new_bboxes =[]
            for bbox, class_label in zip(bboxes, class_labels):
                bbox = list(np.asarray(bbox))
                bbox.append(class_label)
                new_bboxes.append(bbox)
            
            transformed = transform(image=image, masks=list_of_masks, bboxes=new_bboxes, class_labels=class_labels)
            
            transformed_image = transformed['image']
            image = transformed_image
            
            transformed_bboxes = transformed['bboxes']
            final_bboxes = []
            for bbox in transformed_bboxes:
                final_bboxes.append(bbox[:-1])
            transformed_bboxes = torch.as_tensor(final_bboxes, dtype=torch.float32)
            target["boxes"] = transformed_bboxes
            
            transformed_masks = transformed['masks']
            transformed_masks = np.dstack(transformed_masks)
            transformed_masks = torch.as_tensor(np.copy(transformed_masks), dtype=torch.uint8)
            transformed_masks = torch.permute(transformed_masks, (2,0,1))
            target["masks"] = transformed_masks
            
        return Image.fromarray(image), target
    
def get_train_transforms():
    transforms = []
#     transforms.append(RandomRotation(1))
    transforms.append(ToTensor())
    transforms.append(ConvertImageDtype(torch.uint8))
    transforms.append(RandomHorizontalFlip(0.3))
    transforms.append(RandomImageEqualization(0.2))
    transforms.append(RandomGaussianBlur(0.2))
    transforms.append(ConvertImageDtype(torch.float32))
    return Compose(transforms)

def get_val_transforms():
    transforms = []
    transforms.append(ToTensor())
    return Compose(transforms)

# dataset = LiveCellDataset("LiveCellDataset", get_train_transforms())
# image, target = dataset.__getitem__(14)
# print(image.shape, image.dtype)
# print(target['masks'].shape, target['masks'].dtype)
# print('labels', target['labels'], target['labels'].dtype)
# print('boxes:', target['boxes'].shape, target['boxes'].dtype) #correct
# print('iscrowd:', target['iscrowd'], target['iscrowd'].dtype)
# print('image_id:', target['image_id'], target['image_id'].dtype)
# print('area:', target['area'], target['area'].dtype)

In [5]:
class LiveCellDataset(torch.utils.data.Dataset): #torch.utils.data.Dataset
    def __init__(self, root, transforms, annotation_file_path, image_path_path):
        self.root = root
        self.transforms = transforms
        
        # Loading annotations here
        #self.coco_annotation = COCO(annotation_file="../data/TrashCan/instances_train_trashcan.json")
        #self.coco_annotation = COCO(annotation_file="../data/TrashCan/instances_val_trashcan.json")
        self.coco_annotation = COCO(annotation_file=annotation_file_path)
        self.image_path_path = image_path_path
        
        # Loading their ids, please note only one category!
        self.image_ids = self.coco_annotation.getImgIds(catIds=[self.coco_annotation.getCatIds()[0]])

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_info = self.coco_annotation.loadImgs([img_id])[0]
        img_file_name = img_info["file_name"]
        
        #img_path = Path("../data/TrashCan/train") / img_file_name
        #img_path = Path("../data/TrashCan/val") / img_file_name
        
        img_path = Path(self.image_path_path) / img_file_name

        
        img = Image.open(str(img_path)).convert("RGB")
#         display_image(img)
        
        ann_ids = self.coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
        anns = self.coco_annotation.loadAnns(ann_ids)
        
        target = self.get_target(anns, img_info, idx)
    
        if self.transforms is not None:
            img, target = self.transforms(img, target)
            return img, target
        else:
            return ToTensor()(img, target)[0], target


    def get_target(self, annotations, image_info, idx):
        boxes = []
        labels = []
        masks = []
        area = []
        iscrowd = []
        
        image_id = torch.tensor([idx], dtype=torch.int64)
        
        for annotation in annotations:
            one_mask = self.coco_annotation.annToMask(annotation)
            masks.append(one_mask)
            
            labels.append(annotation['category_id'])
            
            bounding_box = annotation['bbox']
            boxes.append(bounding_box)
            
            _iscrowd = annotation['iscrowd']
            iscrowd.append(_iscrowd)
            
            _area = annotation['area']
            area.append(_area)
            
        masks = np.dstack(masks)
        masks = torch.as_tensor(np.copy(masks), dtype=torch.uint8)
        masks = torch.permute(masks, (2,0,1))
        
        labels = torch.as_tensor(labels, dtype=torch.int64)
    
        area = torch.as_tensor(area, dtype=torch.float64)
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.uint8)
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        boxes = box_convert(boxes, in_fmt='xywh', out_fmt='xyxy')
        
        assert len(boxes) == len(iscrowd) == len(labels) == len(area)
        if not (len(boxes) == len(iscrowd) == len(labels) == len(area)):
            print(len(boxes), len(iscrowd), len(labels), len(area))
            
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        return target

    def __len__(self):
        return len(self.image_ids)

# dataset = LiveCellDataset("LiveCellDataset", get_val_transforms(), '../data/TrashCan/instances_train_trashcan.json', 
#                                 "../data/TrashCan/train")
# image, target = dataset.__getitem__(15)
# print(image.shape, image.dtype)
# print(target['masks'].shape, target['masks'].dtype)
# print('labels', target['labels'], target['labels'].dtype)
# print('boxes:', target['boxes'].shape, target['boxes'].dtype) #correct
# print('iscrowd:', target['iscrowd'], target['iscrowd'].dtype)
# print('image_id:', target['image_id'], target['image_id'].dtype)
# print('area:', target['area'], target['area'].dtype)

In [6]:
# Source:
# https://github.com/pytorch/vision/tree/main/references/segmentation
import torch
import torchvision.models.detection.mask_rcnn
import utils_github.utils as utils
from utils_github.coco_eval import CocoEvaluator
from utils_github.coco_utils import get_coco_api_from_dataset
import math
import sys
import time
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq, scaler=None):
    model.train()
#     print("len(data_loader)/print_freq:", len(data_loader)/print_freq)
    metric_logger = utils.MetricLogger(delimiter="  ", epoch=epoch, test=True, 
                                       how_much=int(len(data_loader)/print_freq))
    metric_logger.add_meter("lr", utils.SmoothedValue(window_size=1, fmt="{value:.6f}"))
    header = f"Epoch: [{epoch}]"

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        # print(targets)
        # print(targets.values)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.cuda.amp.autocast(enabled=scaler is not None):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
#             print("Moje", losses)

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())
#         print("Loss_dict_reduced", loss_dict_reduced)
#         print("Losses_reduced", losses_reduced)
        loss_value = losses_reduced.item()
#         print("Loss_value", loss_value)
        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        if scaler is not None:
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            losses.backward()
            optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger


def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types


@torch.inference_mode()
def evaluate(model, data_loader, device, epoch):
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ", epoch=epoch, test=False)
    header = "Test:"

    coco = get_coco_api_from_dataset(data_loader.dataset)
    iou_types = _get_iou_types(model)
    coco_evaluator = CocoEvaluator(coco, iou_types)

    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(device) for img in images)

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
#     print("Averaged stats:", metric_logger)
#     print(metric_logger.meters)
#     wandb.log({"epoch": epoch,"validation_loss": metric_logger})
    coco_evaluator.synchronize_between_processes()

    # accumulate predictions from all images
    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator

In [7]:
def collate_fn(batch):
    return tuple(zip(*batch))

dataset_train = LiveCellDataset("LiveCellDataset", get_train_transforms(), 
                                '../data/TrashCan/instances_train_trashcan.json', 
                                "../data/TrashCan/train")

dataset_test = LiveCellDataset("LiveCellDataset", get_val_transforms(), 
'../data/TrashCan/instances_val_trashcan.json', 
"../data/TrashCan/val")

# indices = torch.randperm(len(dataset_test)).tolist()
# dataset_test = torch.utils.data.Subset(dataset_test, indices[:30])

indices = torch.randperm(len(dataset_train)).tolist()
dataset_train = torch.utils.data.Subset(dataset_train, indices[:2100])
dataset_loader_train = torch.utils.data.DataLoader(
dataset_train, batch_size=1, shuffle=True, num_workers=4, collate_fn=collate_fn)

dataset_loader_test = torch.utils.data.DataLoader(
dataset_test, batch_size=1, shuffle=True, num_workers=4, collate_fn=collate_fn)

# image, target = dataset_test.__getitem__(15)
# print(image.shape, image.dtype)
# print(target['masks'].shape, target['masks'].dtype)
# print('labels', target['labels'], target['labels'].dtype)
# print('boxes:', target['boxes'].shape, target['boxes'].dtype) #correct
# print('iscrowd:', target['iscrowd'], target['iscrowd'].dtype)
# print('image_id:', target['image_id'], target['image_id'].dtype)
# print('area:', target['area'], target['area'].dtype)

loading annotations into memory...
Done (t=0.20s)
creating index...
index created!
loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


In [8]:
wandb.init(project="my-test-project", entity="kolor200")
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

def get_model_instance_segmentation(num_classes):
    backbone = torchvision.models.mobilenet_v2(pretrained=True).features
    backbone.out_channels = 1280
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                       aspect_ratios=((0.5, 1.0, 2.0),))
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                    output_size=7,
                                                    sampling_ratio=2)
    return FasterRCNN(backbone,
                       num_classes=num_classes,
                       rpn_anchor_generator=anchor_generator,
                       box_roi_pool=roi_pooler)


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    # 1 + 22
    num_classes = 23

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)
    # let's train it for 10 epochs
    num_epochs = 10
    print(device)
    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, dataset_loader_train, device, epoch, print_freq=10)
        if epoch % 3 == 0:
            print("Saving model")
            torch.save(model.state_dict(), f"models2/{epoch}.model")
            evaluate(model, dataset_loader_test, device=device, epoch=epoch)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
#         evaluate(model, dataset_loader_test, device=device, epoch=epoch)
        
    
    torch.save(model.state_dict(), f"models2/{num_epochs}.model")
    return model 

model = main()

wandb: Currently logged in as: kolor200 (use `wandb login --relogin` to force relogin)


cuda


/home/janek/anaconda3/envs/yolo6/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180544224/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch: [0]  [   0/2100]  eta: 0:20:02  lr: 0.000010  loss: 4.0023 (4.0023)  loss_classifier: 3.1262 (3.1262)  loss_box_reg: 0.0917 (0.0917)  loss_objectness: 0.6883 (0.6883)  loss_rpn_box_reg: 0.0961 (0.0961)  time: 0.5728  data: 0.1068  max mem: 2445
Epoch: [0]  [  10/2100]  eta: 0:11:50  lr: 0.000060  loss: 3.8177 (3.8511)  loss_classifier: 3.0671 (3.0295)  loss_box_reg: 0.0488 (0.0635)  loss_objectness: 0.6714 (0.6709)  loss_rpn_box_reg: 0.0525 (0.0872)  time: 0.3399  data: 0.0134  max mem: 3004
Epoch: [0]  [  20/2100]  eta: 0:11:24  lr: 0.000110  loss: 3.5867 (3.4754)  loss_classifier: 2.7486 (2.6472)  loss_box_reg: 0.0553 (0.0742)  loss_objectness: 0.6601 (0.6613)  loss_rpn_box_reg: 0.0525 (0.0926)  time: 0.3168  data: 0.0029  max mem: 3005
Epoch: [0]  [  30/2100]  eta: 0:11:15  lr: 0.000160  loss: 2.3524 (2.9007)  loss_classifier: 1.5484 (2.0627)  loss_box_reg: 0.0699 (0.0898)  loss_objectness: 0.6369 (0.6484)  loss_rpn_box_reg: 0.0770 (0.0999)  time: 0.3185  data: 0.0017  max me

Epoch: [0]  [ 330/2100]  eta: 0:08:54  lr: 0.001658  loss: 0.5391 (0.9073)  loss_classifier: 0.1245 (0.3557)  loss_box_reg: 0.0862 (0.0956)  loss_objectness: 0.1501 (0.3027)  loss_rpn_box_reg: 0.0834 (0.1532)  time: 0.3114  data: 0.0018  max mem: 3005
Epoch: [0]  [ 340/2100]  eta: 0:08:52  lr: 0.001708  loss: 0.4550 (0.8937)  loss_classifier: 0.1285 (0.3498)  loss_box_reg: 0.0889 (0.0959)  loss_objectness: 0.1455 (0.2980)  loss_rpn_box_reg: 0.0528 (0.1500)  time: 0.3186  data: 0.0018  max mem: 3005
Epoch: [0]  [ 350/2100]  eta: 0:08:50  lr: 0.001758  loss: 0.4155 (0.8850)  loss_classifier: 0.1681 (0.3459)  loss_box_reg: 0.0934 (0.0967)  loss_objectness: 0.1245 (0.2942)  loss_rpn_box_reg: 0.0261 (0.1482)  time: 0.3211  data: 0.0019  max mem: 3005
Epoch: [0]  [ 360/2100]  eta: 0:08:48  lr: 0.001808  loss: 0.4339 (0.8736)  loss_classifier: 0.1656 (0.3408)  loss_box_reg: 0.1098 (0.0976)  loss_objectness: 0.1132 (0.2895)  loss_rpn_box_reg: 0.0270 (0.1458)  time: 0.3205  data: 0.0018  max me

...

Epoch: [9]  [ 590/2100]  eta: 0:07:28  lr: 0.000005  loss: 0.0784 (0.1649)  loss_classifier: 0.0318 (0.0641)  loss_box_reg: 0.0422 (0.0673)  loss_objectness: 0.0043 (0.0150)  loss_rpn_box_reg: 0.0017 (0.0185)  time: 0.2951  data: 0.0018  max mem: 3007
Epoch: [9]  [ 600/2100]  eta: 0:07:25  lr: 0.000005  loss: 0.0823 (0.1645)  loss_classifier: 0.0320 (0.0639)  loss_box_reg: 0.0432 (0.0672)  loss_objectness: 0.0047 (0.0149)  loss_rpn_box_reg: 0.0043 (0.0185)  time: 0.2970  data: 0.0018  max mem: 3007
Epoch: [9]  [ 610/2100]  eta: 0:07:22  lr: 0.000005  loss: 0.1369 (0.1657)  loss_classifier: 0.0484 (0.0644)  loss_box_reg: 0.0765 (0.0678)  loss_objectness: 0.0047 (0.0149)  loss_rpn_box_reg: 0.0066 (0.0186)  time: 0.2993  data: 0.0017  max mem: 3007
Epoch: [9]  [ 620/2100]  eta: 0:07:19  lr: 0.000005  loss: 0.1369 (0.1666)  loss_classifier: 0.0454 (0.0643)  loss_box_reg: 0.0540 (0.0676)  loss_objectness: 0.0046 (0.0151)  loss_rpn_box_reg: 0.0066 (0.0196)  time: 0.2969  data: 0.0016  max me

Epoch: [9]  [ 920/2100]  eta: 0:05:50  lr: 0.000005  loss: 0.1521 (0.1735)  loss_classifier: 0.0447 (0.0662)  loss_box_reg: 0.0575 (0.0707)  loss_objectness: 0.0110 (0.0160)  loss_rpn_box_reg: 0.0086 (0.0205)  time: 0.2980  data: 0.0016  max mem: 3007
Epoch: [9]  [ 930/2100]  eta: 0:05:47  lr: 0.000005  loss: 0.1053 (0.1739)  loss_classifier: 0.0430 (0.0665)  loss_box_reg: 0.0475 (0.0709)  loss_objectness: 0.0093 (0.0160)  loss_rpn_box_reg: 0.0068 (0.0205)  time: 0.2978  data: 0.0016  max mem: 3007
Epoch: [9]  [ 940/2100]  eta: 0:05:44  lr: 0.000005  loss: 0.1706 (0.1742)  loss_classifier: 0.0605 (0.0667)  loss_box_reg: 0.0643 (0.0711)  loss_objectness: 0.0118 (0.0160)  loss_rpn_box_reg: 0.0067 (0.0204)  time: 0.2981  data: 0.0016  max mem: 3007
Epoch: [9]  [ 950/2100]  eta: 0:05:41  lr: 0.000005  loss: 0.1628 (0.1737)  loss_classifier: 0.0528 (0.0664)  loss_box_reg: 0.0685 (0.0710)  loss_objectness: 0.0113 (0.0159)  loss_rpn_box_reg: 0.0079 (0.0204)  time: 0.2970  data: 0.0016  max me

Epoch: [9]  [1250/2100]  eta: 0:04:12  lr: 0.000005  loss: 0.1247 (0.1760)  loss_classifier: 0.0308 (0.0671)  loss_box_reg: 0.0371 (0.0717)  loss_objectness: 0.0084 (0.0164)  loss_rpn_box_reg: 0.0117 (0.0209)  time: 0.2968  data: 0.0017  max mem: 3007
Epoch: [9]  [1260/2100]  eta: 0:04:09  lr: 0.000005  loss: 0.1128 (0.1757)  loss_classifier: 0.0336 (0.0669)  loss_box_reg: 0.0431 (0.0716)  loss_objectness: 0.0061 (0.0164)  loss_rpn_box_reg: 0.0028 (0.0209)  time: 0.2944  data: 0.0017  max mem: 3007
Epoch: [9]  [1270/2100]  eta: 0:04:06  lr: 0.000005  loss: 0.1142 (0.1759)  loss_classifier: 0.0349 (0.0670)  loss_box_reg: 0.0478 (0.0718)  loss_objectness: 0.0053 (0.0164)  loss_rpn_box_reg: 0.0034 (0.0208)  time: 0.2947  data: 0.0017  max mem: 3007
Epoch: [9]  [1280/2100]  eta: 0:04:03  lr: 0.000005  loss: 0.1874 (0.1772)  loss_classifier: 0.0711 (0.0676)  loss_box_reg: 0.0810 (0.0722)  loss_objectness: 0.0104 (0.0165)  loss_rpn_box_reg: 0.0095 (0.0208)  time: 0.2970  data: 0.0017  max me

Epoch: [9]  [1580/2100]  eta: 0:02:34  lr: 0.000005  loss: 0.1073 (0.1793)  loss_classifier: 0.0311 (0.0691)  loss_box_reg: 0.0536 (0.0729)  loss_objectness: 0.0055 (0.0168)  loss_rpn_box_reg: 0.0061 (0.0205)  time: 0.2968  data: 0.0017  max mem: 3007
Epoch: [9]  [1590/2100]  eta: 0:02:31  lr: 0.000005  loss: 0.1091 (0.1789)  loss_classifier: 0.0335 (0.0689)  loss_box_reg: 0.0515 (0.0728)  loss_objectness: 0.0055 (0.0167)  loss_rpn_box_reg: 0.0041 (0.0205)  time: 0.2954  data: 0.0017  max mem: 3007
Epoch: [9]  [1600/2100]  eta: 0:02:28  lr: 0.000005  loss: 0.1080 (0.1787)  loss_classifier: 0.0430 (0.0689)  loss_box_reg: 0.0423 (0.0728)  loss_objectness: 0.0076 (0.0167)  loss_rpn_box_reg: 0.0028 (0.0204)  time: 0.2936  data: 0.0016  max mem: 3007
Epoch: [9]  [1610/2100]  eta: 0:02:25  lr: 0.000005  loss: 0.1202 (0.1785)  loss_classifier: 0.0430 (0.0688)  loss_box_reg: 0.0423 (0.0726)  loss_objectness: 0.0116 (0.0168)  loss_rpn_box_reg: 0.0028 (0.0204)  time: 0.2957  data: 0.0016  max me

Epoch: [9]  [1910/2100]  eta: 0:00:56  lr: 0.000005  loss: 0.1181 (0.1784)  loss_classifier: 0.0342 (0.0689)  loss_box_reg: 0.0534 (0.0730)  loss_objectness: 0.0090 (0.0168)  loss_rpn_box_reg: 0.0057 (0.0196)  time: 0.2976  data: 0.0016  max mem: 3007
Epoch: [9]  [1920/2100]  eta: 0:00:53  lr: 0.000005  loss: 0.1236 (0.1784)  loss_classifier: 0.0403 (0.0689)  loss_box_reg: 0.0479 (0.0729)  loss_objectness: 0.0072 (0.0168)  loss_rpn_box_reg: 0.0086 (0.0197)  time: 0.2981  data: 0.0016  max mem: 3007
Epoch: [9]  [1930/2100]  eta: 0:00:50  lr: 0.000005  loss: 0.1397 (0.1785)  loss_classifier: 0.0476 (0.0689)  loss_box_reg: 0.0490 (0.0729)  loss_objectness: 0.0119 (0.0168)  loss_rpn_box_reg: 0.0101 (0.0198)  time: 0.2963  data: 0.0016  max mem: 3007
Epoch: [9]  [1940/2100]  eta: 0:00:47  lr: 0.000005  loss: 0.1105 (0.1786)  loss_classifier: 0.0382 (0.0689)  loss_box_reg: 0.0477 (0.0729)  loss_objectness: 0.0086 (0.0168)  loss_rpn_box_reg: 0.0042 (0.0199)  time: 0.2971  data: 0.0016  max me

In [9]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
loss,▃▃▃▃▇▃▂▂▁▃▂▂▁▂▅█▂▂▁▁▄▂▄▁▃▂▁▁▃▂▅▂▂▁▅▂▃▄▁▁
loss_box_reg,▃▂▄▃▄▂▁▃▂▂▃▃▁▂██▂▂▂▁▆▃▅▁▃▂▂▁▄▃▇▃▃▁▄▃▄▅▁▁
loss_classifier,▃▃▃▃▅▂▂▂▂▂▃▃▁▂▄█▂▂▁▁▇▂▇▁▂▂▂▁▃▃▆▂▃▁▅▄▄▅▁▁
loss_objectness,▆▅▃▅▇▃▂▂▂▄▃▂▁▂▇█▁▂▁▁▂▁▄▁▄▁▁▁▂▁▂▂▂▁▂▁▃▄▁▁
loss_rpn_box_reg,▁▂▁▂█▃▁▁▁▃▁▁▁▁▂▇▁▁▁▁▁▁▁▁▃▁▁▁▂▁▃▂▁▁▅▁▂▂▁▁
lr,▃▅██████████▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,9
loss,0.11636
loss_box_reg,0.0737
loss_classifier,0.02547


In [12]:
# model = get_model_instance_segmentation(23)
# model.load_state_dict(torch.load("models/10.model"))
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# epoch = 0
# model.to(device)
# evaluate(model, dataset_loader_test, device=device, epoch=epoch)
# # model.eval()